In [3]:
import os
import numpy as np
import sys

MODULE_FULL_PATH = '/home/zuongthao/GIT/zuongthaotn/backtesting.py'
sys.path.insert(1, MODULE_FULL_PATH)
from backtesting import Backtest, Strategy
from backtesting.magnus import get_pricing
from backtesting.test import SMA
from backtesting.lib import crossover
import method.Ticker as Ticker
import method.TakeProfit as TakeProfit
import method.CutLoss as CutLoss

In [11]:
path = os.getcwd()
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        if len(self.data.Volume) > 5:
            last_price = prices[-1]
            if self.buy_price == 0 and Ticker.isFollowTrendingV2(prices, volumes, opens[-1], 2.5):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and (TakeProfit.takeProfit(5, last_price, self.buy_price) or CutLoss.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [12]:
ticker_id = 'PNJ'
ticker = get_pricing(ticker_id+'.csv', '2018-01-01', '2020-10-05')
bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
stats = bt.run()

In [13]:
stats

Start                     2018-01-02 00:00:00
End                       2020-10-05 00:00:00
Duration                   1007 days 00:00:00
Exposure Time [%]                     9.46143
Equity Final [$]                      12027.5
Equity Peak [$]                         12041
Return [%]                             20.275
Buy & Hold Return [%]                -3.26894
Return (Ann.) [%]                     7.00629
Volatility (Ann.) [%]                 8.94714
Sharpe Ratio                         0.783076
Sortino Ratio                         1.47626
Calmar Ratio                         0.746367
Max. Drawdown [%]                     -9.3872
Avg. Drawdown [%]                    -2.36054
Max. Drawdown Duration      266 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                    4
Win Rate [%]                              100
Best Trade [%]                        5.35556
Worst Trade [%]                        4.0679
Avg. Trade [%]                    

In [14]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,135,151,186,73.694640,77.6414,532.812600,0.053556,2018-08-15,2018-10-04,50 days
1,140,335,338,75.122544,78.4467,465.381840,0.044250,2019-05-16,2019-05-21,5 days
2,146,383,392,74.947071,78.7826,559.987234,0.051177,2019-07-23,2019-08-05,13 days
3,138,492,506,83.602432,87.0033,469.319715,0.040679,2019-12-24,2020-01-14,21 days
